In [1]:
from datetime import datetime, timedelta
import numpy as np
import random
from train import Train
from infrastructure import Track, Block, BigBlock
from signaling import AutoSignal, HomeSignal, AutoPoint, ControlPoint

In [ ]:
_nodes = {  0: ControlPoint(idx=0, ports=[0,1]), \
                    1: AutoPoint(1), \
                    2: AutoPoint(2), \
                    3: ControlPoint(idx=3, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
                    4: ControlPoint(idx=4, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
                    5: AutoPoint(5), \
                    6: ControlPoint(idx=6, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
                    7: ControlPoint(idx=7, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
                    8: AutoPoint(8), \
                    9: AutoPoint(9), \
                    10: ControlPoint(idx=10, ports=[0,1])}

In [1]:
print(1)

1


In [2]:
_nodes

NameError: name '_nodes' is not defined

In [3]:
_nodes = {  0: ControlPoint(idx=0, ports=[0,1]), \
            1: AutoPoint(1), \
            2: AutoPoint(2), \
            3: ControlPoint(idx=3, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
            4: ControlPoint(idx=4, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
            5: AutoPoint(5), \
            6: ControlPoint(idx=6, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
            7: ControlPoint(idx=7, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
            8: AutoPoint(8), \
            9: AutoPoint(9), \
            10: ControlPoint(idx=10, ports=[0,1])}

NameError: name 'ControlPoint' is not defined

In [4]:
from datetime import datetime, timedelta
import numpy as np
import random
from train import Train
from infrastructure import Track, Block, BigBlock
from signaling import AutoSignal, HomeSignal, AutoPoint, ControlPoint

In [ ]:
_nodes = {  0: ControlPoint(idx=0, ports=[0,1]), \
            1: AutoPoint(1), \
            2: AutoPoint(2), \
            3: ControlPoint(idx=3, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
            4: ControlPoint(idx=4, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
            5: AutoPoint(5), \
            6: ControlPoint(idx=6, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
            7: ControlPoint(idx=7, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
            8: AutoPoint(8), \
            9: AutoPoint(9), \
            10: ControlPoint(idx=10, ports=[0,1])}

In [1]:
from datetime import datetime, timedelta
import numpy as np
import random
from train import Train
from infrastructure import Track, Block, BigBlock
from signaling import AutoSignal, HomeSignal, AutoPoint, ControlPoint

In [ ]:
ControlPoint(idx=0, ports=[0,1])

In [1]:
_nodes = {  0:ControlPoint(idx=0, ports=[0,1]), \
                    1:AutoPoint(1), \
                    2:AutoPoint(2), \
                    3:ControlPoint(idx=3, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
                    4:ControlPoint(idx=4, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
                    5:AutoPoint(5), \
                    6:ControlPoint(idx=6, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
                    7:ControlPoint(idx=7, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
                    8:AutoPoint(8), \
                    9:AutoPoint(9), \
                    10:ControlPoint(idx=10, ports=[0,1])}

NameError: name 'ControlPoint' is not defined

In [2]:
from datetime import datetime, timedelta
import numpy as np
import random
from train import Train
from infrastructure import Track, Block, BigBlock
from signaling import AutoSignal, HomeSignal, AutoPoint, ControlPoint

In [ ]:
_nodes = {  0:ControlPoint(idx=0, ports=[0,1]), \
                    1:AutoPoint(1), \
                    2:AutoPoint(2), \
                    3:ControlPoint(idx=3, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
                    4:ControlPoint(idx=4, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
                    5:AutoPoint(5), \
                    6:ControlPoint(idx=6, ports=[0,1,3], ban_routes={1:[3],3:[1]}), \
                    7:ControlPoint(idx=7, ports=[0,2,1], ban_routes={0:[2],2:[0]}), \
                    8:AutoPoint(8), \
                    9:AutoPoint(9), \
                    10:ControlPoint(idx=10, ports=[0,1])}

In [1]:
AutoPoint(1)

NameError: name 'AutoPoint' is not defined

In [2]:
from datetime import datetime, timedelta
import numpy as np
import random
from train import Train
from infrastructure import Track, Block, BigBlock
from signaling import AutoSignal, HomeSignal, AutoPoint, ControlPoint

In [3]:
AutoPoint(1)

In [ ]:
ControlPoint(idx=10, ports=[0,1])

In [1]:
from signaling import AutoSignal, HomeSignal, AutoPoint, ControlPoint

In [2]:
import networkx as nx

In [ ]:
ControlPoint(idx=3, ports=[0,1,3], ban_routes={1:[3],3:[1]})

In [1]:
from datetime import datetime, timedelta
import numpy as np
import random
from train import Train
from infrastructure import Track, Block, BigBlock
from signaling import AutoSignal, HomeSignal, AutoPoint, ControlPoint

ImportError: bad magic number in 'train': b'\x03\xf3\r\n'

In [2]:
from datetime import datetime, timedelta
import numpy as np
import random
from train import Train
from infrastructure import Track, Block, BigBlock
from signaling import AutoSignal, HomeSignal, AutoPoint, ControlPoint

In [ ]:
ControlPoint(idx=3, ports=[0,1,3], ban_routes={1:[3],3:[1]})